## Dynamic Plans

Here, instead of a predefined plan, vehicles take decisions online following a minimization distance rule. Only a goal is predefined.

In [1]:
import os
import numpy as np

# Custom modules
from helpers.change_coordinates import global2local
from helpers.visualization.plots import plot_3d_interactive
from plan import Plan,State
from gcs import GCS
from simulators.QGroundControl.qgc import QGC
from simulators.gazebo.gazebo import Gazebo

from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint

#Kill all related process
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Gonfiguration

In [2]:
offsets = [ #east, north, up, heading
    (40, -10, 0, 0), 
    (40,  20, 0, 0)]
n_vehicles = len(offsets)

wps = np.array([(40, -10, 5),
                (15, -10, 5),
                (0, -10, 5),
                (0,   4, 5),
                (40, 20, 5),
                (15, 20, 5),
                (0,  20, 5)])

homes=np.array([offset[:3] for offset in offsets])
target_wps=np.array([
                (0, 4,5),
                (0, 4,5)])

targets=np.array([(*target_wp[:2],0) for target_wp in target_wps])
plans=[None,None]

## Visualization Parameters

In [3]:
markers = {'waypoints': {'pos':wps,'color':'yellow'},
           'homes': {'pos':homes,'color':'blue'},
           'targets':{'pos':targets,'color':'green'}}

# Interactive widget to rotate the plot
plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)

## Waypoint selection algorithm

The path is shown in local coordinates

In [4]:
local_wps=global2local(wps, homes,pairwise=False)
local_target_wps=global2local(target_wps, homes,pairwise=True)


for i,(target_wp,wps) in enumerate(zip(local_target_wps,local_wps)):
        local_path=find_path(np.array((0,0,0)),target_wp, wps,eps=1)
        print(f'uav-{i} Best path: {local_path}') 


uav-0 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40  14   5]]
uav-1 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40 -16   5]]


## Choose Simulator

In [5]:
simulator=QGC(offsets=offsets,
              plans=plans,
              origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path="simulators/gazebo/worlds/small_city.world",
                 vehicle_models=n_vehicles*["drone"],
                 waypoints=markers)

Launch Simulator

In [6]:
uavs = simulator.launch()

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is no

## Execute Plan (Not yet updated to reflect the new refactoring)

In [7]:
from plan.core import State
from plan.actions.navegation import get_local_position
from plan.actions.navegation import make_go_to


In [8]:
gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        i = id-1
        uav=uavs[i]
        ## It is better to define this as a plan
        curr_action=uav.current_action()
        if curr_action.name=="fly" and (curr_action.state == State.NOT_STARTED or (curr_action.state == State.IN_PROGRESS and curr_action.current.state == State.DONE)):
            ## Search algorithm
            final_wp = local_target_wps[i]
            curr_pos = get_local_position(uav.conn)
            if curr_pos is False:
                continue
            dist = np.linalg.norm(curr_pos - final_wp)
            if dist > 0.5:
                next_wp = find_best_waypoint(curr_pos, final_wp, local_wps[i],eps=1)
                next_step=make_go_to(wp=next_wp,wp_margin=0.5)
                uav.plan.current.add_next(next_step)
            uav.plan.act()
        else:
            uav.plan.act()
        if uav.plan.state==State.DONE:
            to_remove.add(id)
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: arm is done
Vehicle 1: ▶️ Starting Step: takeoff
Vehicle 1:📍 Distance to target: 5.04 m
Vehicle

AttributeError: 'NoneType' object has no attribute 'target_system'

## Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")